# Welcome to the Prototype

Table of Contents:
1) **Introduction**<br>
2) **Datasets**<br>
    2.1) Synthetic Dataset<br>
    2.2) 30 Years of European Wind Generation <br>
3) **Transformer Architectures**<br>
    3.1) Informer<br>
    3.2) Autoformer<br>
    3.4) Pyraformer<br>
4) **Forecasting**<br>
    4.1) 24-Hour Windows (Day)<br>
    4.2) 168-Hour Windows (Week)<br>
    4.3) 720-Hour Windows (Month)<br>
5) **Summary**

## **1. Introduction**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')

## **2. Datasets**

### **2.1) Synthetic Data**

In [ ]:
from data_generator import SynthesisTS

In [ ]:
# Initialize data synthesizer
SynthMaker = SynthesisTS(series_amount=1, seq_len=17420)

In [ ]:
# Generate first synthetic time series
df1 = SynthMaker.synthesize_single_series()

### **2.2) 30 Years of European Wind Generation**

## **3. Transformer Architectures**

### **3.1) Informer**

### **3.2) Autoformer**

### **3.4) Pyraformer**

## **4. Forecasting**

### **4.1) 24-Hour Windows (Day)**

### **4.2) 168-Hour Windows (Week)**

### **4.3) 720-Hour Windows (Month)**

## **5. Summary**